In [31]:
import os
import numpy as np
from keras.optimizers import RMSprop
from keras.losses import categorical_crossentropy

In [32]:
RND = 0
RUN = 'A'
OUT_DIR = 'out/%s' % RUN

In [33]:
%run 'lib.ipynb'
%run 'data-generator.ipynb'
%run 'models.ipynb'

In [34]:
np.random.seed(RND)
if not os.path.isdir(OUT_DIR): os.makedirs(OUT_DIR)

In [19]:
# create model
model = Model_1(classes=LABELS)
model.build()
optimizer = RMSprop(lr=0.001, decay=0.0)
model.m.compile(
    optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy']\
)